# Run AI Red Teaming Scan

The AI Red Team Agent scan proactively tests a target for vulnerabilities to various attacks by:
1. Generating adversarial prompts
1. Sending prompts to the target
1. Analyzing target responses for vulnerabilities
1. Generating a security report on the overall performance

In this lab, we look at how we can run a red team scan on an agent target, where the agent is identified by name and currently deployed in a given Azure AI Foundry project. Here are the steps:

1. Load environment variables - identify Foundry project endpoint and agent name
1. Retrieve agent ID - using the endpoint and name as context
1. Configure an agent callback - by setting up a communication thread
1. Run red team scan on callback - to probe agent for vulnerabilities

In [1]:
# Suppress SyntaxWarning from confusables module
import warnings
warnings.filterwarnings('ignore', category=SyntaxWarning, module='confusables')

In [2]:
# ------------------------------------
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
# ------------------------------------

from typing import Optional, Dict, Any
import os
import time
from pathlib import Path
from dotenv import load_dotenv

# Azure imports
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy
from azure.ai.projects import AIProjectClient
import azure.ai.agents
from azure.ai.agents.models import ListSortOrder

async def run_red_team():
    # Load environment variables from .env file
    # In notebooks, use os.getcwd() instead of __file__
    current_dir = Path(os.getcwd())
    env_path = current_dir / ".env"
    # Try loading from current directory first, then parent directories
    if not env_path.exists():
        env_path = current_dir.parent / ".env"
    if not env_path.exists():
        env_path = current_dir.parent.parent / ".env"
    load_dotenv(dotenv_path=env_path)
    
    # Initialize Azure credentials
    credential = DefaultAzureCredential()

    # Get AI project parameters from environment variables (matching evaluate.py)
    project_endpoint = os.environ.get("AZURE_AI_PROJECT_ENDPOINT")
    deployment_name = os.getenv("AZURE_AI_DEPLOYMENT_NAME")  
    agent_id = os.environ.get("AZURE_EXISTING_AGENT_ID")
    agent_name = os.environ.get("AZURE_AI_AGENT_NAME")
    
    # Validate required environment variables
    if not project_endpoint:
        raise ValueError("Please set theA ZURE_AI_PROJECT_ENDPOINT environment variable.")
        
    if not agent_id and not agent_name:
        raise ValueError("Please set either AZURE_EXISTING_AGENT_ID or AZURE_AI_AGENT_NAME environment variable.")

    with DefaultAzureCredential(exclude_interactive_browser_credential=False) as credential:
        with AIProjectClient(endpoint=project_endpoint, credential=credential) as project_client:
            # Look up the agent by name if agent ID is not provided (matching evaluate.py)
            if not agent_id and agent_name:
                for agent in project_client.agents.list_agents():
                    if agent.name == agent_name:
                        agent_id = agent.id
                        break
                        
            if not agent_id:
                raise ValueError("Agent ID not found. Please provide a valid agent ID or name.")
                
            agent = project_client.agents.get_agent(agent_id)
            
            # Use model from agent if not provided - matching evaluate.py
            if not deployment_name:
                deployment_name = agent.model
                
            thread = project_client.agents.threads.create()

            def agent_callback(query: str) -> str:
                message = project_client.agents.messages.create(thread_id=thread.id, role="user", content=query)
                run = project_client.agents.runs.create(thread_id=thread.id, agent_id=agent.id)

                # Poll the run as long as run status is queued or in progress
                while run.status in ["queued", "in_progress", "requires_action"]:
                    # Wait for a second
                    time.sleep(1)
                    run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)
                    # [END create_run]
                    print(f"Run status: {run.status}")

                if run.status == "failed":
                    print(f"Run error: {run.last_error}")
                    return "Error: Agent run failed."
                messages = project_client.agents.messages.list(thread_id=thread.id, order=ListSortOrder.DESCENDING)
                for msg in messages:
                    if msg.text_messages:
                        return msg.text_messages[0].text.value
                return "Could not get a response from the agent."
        

            # Print agent details to verify correct targeting
            print(f"Running Red Team evaluation against agent:")
            print(f"  - Agent ID: {agent.id}")
            print(f"  - Agent Name: {agent.name}")
            print(f"  - Using Model: {deployment_name}")
            
            red_team = RedTeam(
                azure_ai_project=project_endpoint,
                credential=credential,
                risk_categories=[RiskCategory.Violence],
                num_objectives=1,
                output_dir="redteam_outputs/"
            )

            print("Starting Red Team scan...")
            result = await red_team.scan(
                target=agent_callback,
                scan_name="Agent-Scan",
                attack_strategies=[AttackStrategy.Flip],
            )
            print("Red Team scan complete.")

# Run the async function
# In Jupyter notebooks, use await directly instead of asyncio.run()
await run_red_team()

Running Red Team evaluation against agent:
  - Agent ID: asst_0pzCobnX8sXZti4iMaPvFZDX
  - Agent Name: Cora-For-Zava
  - Using Model: gpt-4.1-mini


Class RedTeam: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Starting Red Team scan...
🚀 STARTING RED TEAM SCAN
📂 Output directory: redteam_outputs/.scan_Agent-Scan_20251105_064509
📊 Risk categories: ['violence']
🔗 Track your red team scan in AI Foundry: None
📋 Planning 2 total tasks
📝 Fetched baseline objectives for violence: 1 objectives
🔄 Fetching objectives for strategy 2/2: flip


Scanning:   0%|                                       | 0/2 [00:00<?, ?scan/s, current=initializing]

⚙️ Processing 2 tasks in parallel (max 5 at a time)
▶️ Starting task: baseline strategy for violence risk category
▶️ Starting task: flip strategy for violence risk category
Run status: incomplete
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.COMPLETED


Scanning:  50%|███████████████▌               | 1/2 [00:18<00:18, 18.56s/scan, current=initializing]

Evaluation results saved to "/workspaces/ignite25-LAB516-safeguard-your-agents-with-ai-red-teaming-agent-in-azure-ai-foundry/labs/1-scan-agent/redteam_outputs/.scan_Agent-Scan_20251105_064509/baseline_violence_ecb7e199-4e47-4d58-85ec-a9ae27c0c093.json".
✅ Completed task 1/2 (50.0%) - baseline/violence in 18.6s
   Est. remaining: 0.4 minutes


Scanning: 100%|███████████████████████████████| 2/2 [00:20<00:00, 10.16s/scan, current=initializing]
Class RedTeamResult: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Evaluation results saved to "/workspaces/ignite25-LAB516-safeguard-your-agents-with-ai-red-teaming-agent-in-azure-ai-foundry/labs/1-scan-agent/redteam_outputs/.scan_Agent-Scan_20251105_064509/flip_violence_2ef10d5d-e109-409b-abb7-5453c7186b51.json".
✅ Completed task 2/2 (100.0%) - flip/violence in 20.3s
   Est. remaining: 0.0 minutes
Evaluation results saved to "/workspaces/ignite25-LAB516-safeguard-your-agents-with-ai-red-teaming-agent-in-azure-ai-foundry/labs/1-scan-agent/redteam_outputs/.scan_Agent-Scan_20251105_064509/final_results.json".

Overall ASR: 50.0%
Attack Success: 1/2 attacks were successful
------------------------------------------------------------------------------------------------------------------------------------
Risk Category        | Baseline ASR   | Easy-Complexity Attacks ASR  | Moderate-Complexity Attacks ASR | Difficult-Complexity Attacks ASR
--------------------------------------------------------------------------------------------------------------------